# imbed_L1_L2_Syndigo Mapping ML
  - module name: imbed_L1_L2_Syndigo ML.ipynb
  - Purpose: Combine Level 1 & Level 2 from  Syndigo to build model
             - Apply TFIDF and W2vec imbedding methods 

In [1]:
def prepare_data_1():
    pgm = inspect.currentframe().f_code.co_name  
    start_time = time.time() 
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    #print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    syndigo_mapped['l1_l2'] = syndigo_mapped['Level 1'] + ' + ' + syndigo_mapped['Level 2']
    #print(f"syndigo_mapped['l1_l2'].value_counts() {syndigo_mapped['l1_l2'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [24]:
def prepare_data_2(samp_frac = 1, embed_typ = 'tfidf'):
  
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    global syndigo_mapped, model
    print(f' samp_frac = {samp_frac}') 
    print (f'Before sample syndigo_mapped.shape {syndigo_mapped.shape }')
    syndigo_mapped = syndigo_mapped_bkup.copy()
    
    whole_frac = 1
    if samp_frac  < whole_frac :   syndigo_mapped = syndigo_mapped.sample(frac = samp_frac,  random_state=42)   
    print (f'After sample syndigo_mapped.shape {syndigo_mapped.shape }')    
    
   
    if embed_typ == 'tfidf': 
        vect = TfidfVectorizer(ngram_range = (1,2), max_features = 50000) 
        X = vect.fit_transform(syndigo_mapped.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    else:
        model = KeyedVectors.load_word2vec_format(DBFO + 'w2vmodel_053123_PIM_ALL.bin', binary=True)
        X = np.array(list(syndigo_mapped.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))

    l1_l2_id_map = dict(zip(syndigo_mapped['l1_l2'].fillna('Other').unique(), range(syndigo_mapped['l1_l2'].fillna('Other').nunique())))
    id_l1_l2_map = dict(zip(range(syndigo_mapped['l1_l2'].fillna('Other').nunique()), syndigo_mapped['l1_l2'].fillna('Other').unique()))
    y  = syndigo_mapped['l1_l2'].fillna('Other').map(l1_l2_id_map)
    
    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return  X, y, l1_l2_id_map, id_l1_l2_map

In [22]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [4]:
def proc_predict(test_size,c_parm, embed_typ):
     start_time = time.time()
     pgm = inspect.currentframe().f_code.co_name   
     global A_train, A_test, B_train, B_test, X_train, X_test, y_train, y_test
     A_train, A_test, B_train, B_test = train_test_split(syndigo_mapped.GTIN.tolist(), syndigo_mapped['l1_l2'].tolist(), test_size=test_size,  random_state=42)
     
     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size,  random_state=42)
     lr_tf = LogisticRegression(C = c_parm, multi_class = 'multinomial', solver = 'saga', n_jobs=-1)
     
     print(f"Training starts for test_size = {test_size}, c_parm = {c_parm} ")
     lr_tf.fit(X_train, y_train)
     preds = lr_tf.predict(X_test)
     #print(f"type(preds) {type(preds)} len(preds) {len(preds)} \n preds[0:10] {preds[0:10]}")
     preds_lrtf = preds
     probs = lr_tf.predict_proba(X_test)
     #print(f"type(probs) {type(probs)} len(probs) {len(probs)} \n probs[0:10] {probs[0:10]}")   
     preds_train = lr_tf.predict(X_train)
     #print(f"type(preds_train) {type(preds_train)} len(preds_train) {len(preds_train)} \n preds_train[0:10] {preds_train[0:10]} ")   
     probs_train = lr_tf.predict_proba(X_train)
     #print(f"type(probs_train) {type(probs_train)} len(probs_train) {len(probs_train)} \n probs_train[0:10] {probs_train[0:10]}")   
     #print(classification_report(y_test, preds_test,labels = lr_tf.classes_, target_names = [id2_level_map[i] for i in lr_tf.classes_]))
     
    
    
     #print("Accuracy = ",accuracy_score(y_test,preds))
     #print("Display TFIDF metrics")
     #print(classification_report(y_test, preds,labels = lr_tf.classes_, target_names = [id_l1_l2_map[i] for i in lr_tf.classes_]))
     filename = path + f'L1_l2_LR_{embed_typ}_tst_siz{test_size}_C{c_parm}.pkl'
   
     pickle.dump(lr_tf, open(filename, 'wb'))

     print(f"Done Training ")
     end_time = time.time()   
     desc = f' Elapse_time for "{pgm}".' 
     elapse_time (  start_time, end_time, desc)
     return preds, preds_lrtf, probs, preds_train, probs_train, lr_tf

In [5]:
def validation(test_size, c_parm, embed_typ):
     start_time = time.time() 
     pgm = inspect.currentframe().f_code.co_name
     global f1_score_weighted, f1_score_macro,f1_score_micro 
     l_metrics = []
     print(f'Validation for c_parm = {c_parm}, test_size = {test_size}, and embed_type {embed_typ} ' ) 
     accuracy = accuracy_score(y_test,preds)
     print(f"Accuracy Score = {accuracy}")
     
     f1_score_weighted = f1_score(y_test, preds, average="weighted")
     f1_score_macro    = f1_score(y_test, preds, average='macro')
     f1_score_micro    = f1_score(y_test, preds, average='micro')  
     print (f" f1_score_weighted = {f1_score_weighted}")
     print (f" f1_score_macro    = {f1_score_macro   }")
     print (f" f1_score_micro    = {f1_score_micro   }")
     # Compute the Multiclass ROC AUC score
     #multi_class = 'multinomial'
     #score = roc_auc_score(y_test, preds, multi_class= multi_class)
     #print(f"ROC AUC score for Multiclass= {multi_class}: {score:.2f}")

     print("Display metrics")
     test_metrics = pd.DataFrame(classification_report(y_test, preds,labels = lr_tf.classes_, target_names = [id_l1_l2_map[i] for i in lr_tf.classes_],  output_dict= True)).T
     #test_metrics = pd.DataFrame(classification_report(results_test.Actuals, results_test.Predictions, output_dict= True)).T 
     #print(test_metrics)
     #Stest_metrics.to_csv('L1_L2_Test_Metrics_yue.csv')
     #accuracy = 1
     tab_name = f'{embed_typ}_L1_L2_TM_tst_siz{test_size}_C{c_parm}'   
     print(f'tab_name = {tab_name}')  
     l_metrics = [embed_typ, test_size, c_parm, accuracy, f1_score_weighted, f1_score_macro,f1_score_micro ]
     end_time = time.time()   
     desc = f' Elapse_time for "{pgm}".' 
     elapse_time (  start_time, end_time, desc)    
     return test_metrics, tab_name, l_metrics 
                               

In [6]:
def proc_append():
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name     
    test_new_proobs = []
    
    #print(f'len(preds)  {len(preds)}')
    for i in range(len(preds)):
        test_new_proobs.append(probs[i][np.argsort(probs[i])][::-1][:1].tolist())
    test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]
    #print(f'test_new_proobs {test_new_proobs}')
    
    train_new_proobs = []
    
    #print(f'len(preds_train)  {len(preds_train)}')
    for i in range(len(preds_train)):
         test_new_proobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1].tolist())
    train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]
    #print(f'train_new_proobs {train_new_proobs}')  
    
    # process testLevels & trainLevelss 
    testLevels = []
    for j in y_test:
        testLevels.append([i for i in l1_l2_id_map if l1_l2_id_map[i]==j][0])
    #print(f'testLevels {testLevels [0:10]}')
    testLevelss = []
    for j in preds:
        testLevelss.append([i for i in l1_l2_id_map if l1_l2_id_map[i]==j][0])
    #print(f'testLevelss {testLevelss [0:10]}')
    
    trainLevelss = []
    for j in preds_train:
        trainLevelss.append([i for i in l1_l2_id_map if l1_l2_id_map[i]==j][0])
     
    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)    
    return  test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss

In [7]:
def proc_cr_df(test_size, c_parm, embed_typ):

    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
    print("1)", len(A_test + A_train))
    print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
    print("3)", len(B_test + B_train))
    print("4)", len(testLevels + B_train))
    print("5)", len(testLevelss + trainLevelss))
    print("6)", int(len(test_new_proobs + train_new_proobs)))
    
    data = {
        'GTIN' : A_test + A_train,
        'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
        'Actual l1_l2' : B_test + B_train,
        'Actuals' : testLevels + B_train,
        'Predictions' : testLevelss + trainLevelss,
        'Scores' : test_new_proobs + train_new_proobs,
    
    }
    
    df = pd.DataFrame(data)
    df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
        'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
        'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
    df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
        'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
        'VND_ECOM_DSC', 'Source', 'Actual l1_l2', 'Actuals', 'Predictions', 'Scores']]
    #filename = 'Syndigo_Mapping_L1_L2_ML.csv'
    tab_name = f'{embed_typ}_L1_L2_prd_tst_siz{test_size}_C{c_parm}'  
    #df.to_csv(DBFO + filename, index=False)
    #print(f"{filename}  --file created successfully")

    end_time = time.time()   
    desc = f'Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return df, tab_name

In [9]:
def save_excel_pred( excel_file, dict):
#  Write classification report to excel
#  save() is going to remove, use close() instead
   from pandas import ExcelWriter
   from pandas import ExcelFile
   writer = pd.ExcelWriter(excel_file)
   for key,  value in dict.items():
       df   =  value[0] 
       tab  =  value[1] 
       df.to_excel(writer,tab)
   writer.close()
   return

In [28]:
def save_excel_metric( excel_file, dict, l_metric):
#  Write classification report to excel
#  save() is going to remove, use close() instead
   from pandas import ExcelWriter
   from pandas import ExcelFile
   writer = pd.ExcelWriter(excel_file)
   for key,  value in dict.items():
       df   =  value[0] 
       tab  =  value[1] 
       df.to_excel(writer,tab)
   df_pred = pd.DataFrame(l_metric, columns = ['embed_typ', 'test_size', 'c_parm', 'accuracy', 'f1_score_weighted', 'f1_score_macro',
                                                'f1_score_micro', 'elapse_time'] )   
   df_pred.to_excel(writer,'Consolidate Metrics')                                         
   writer.close()
   return

In [30]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from datetime import date


from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
import re, time, inspect, pickle
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))


In [12]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [13]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()
syndigo_mapped_bkup = syndigo_mapped.copy()


In [37]:
l_c_parm = [0.1, 1, 10, 100] 
l_embed_typ = ['tfidf', 'W2vec']

dic_metric={}
dic_predict = {}
l_metric=[]
test_size = 0.3
i = 0
for embed_typ in l_embed_typ:
    X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 1, embed_typ = embed_typ )
    for c_parm in l_c_parm:
        start_time = time.time()        
        
        preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3, c_parm =c_parm,  embed_typ= embed_typ )
 
        test_metrics, vtab, l_metrics  =  validation(test_size = 0.3, c_parm =c_parm, embed_typ=embed_typ) 
        key = 'test_metric' + str(i)
        dic_metric [key] = (test_metrics, vtab)
        
        test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss = proc_append()
        df_save, ftab  = proc_cr_df(test_size=0.3, c_parm=c_parm, embed_typ= embed_typ )
        key = 'predict'+ str(i)
        dic_predict[key] = (df_save, ftab)
        i = i + 1   
        end_time = time.time() 
        desc = f' Elapse_time to build model for for test_size = 0.3, c_parm = {c_parm}, embed_typ= {embed_typ}' 
        hh, mm, ss= elapse_time (  start_time, end_time, desc)
                                 
        l_metrics = l_metrics +  [f"{hh:2d}hh:{mm:2d}mm:{ss:2d}ss"]   
        l_metric.append(l_metrics)
dte = date.today().strftime('%m%d%y')                                  
excel_file = DBFO + f'L1_L2_syndigo_pred_cls_rpt_size{test_size}_{dte}.xlsx'
save_excel_metric( excel_file, dic_metric, l_metric)


excel_file = DBFO + f'L1_L2_syndigo_pred_tst_size{test_size}_{dte}.xlsx'
save_excel_pred( excel_file, dic_predict)

 samp_frac = 1
Before sample syndigo_mapped.shape (77726, 25)
After sample syndigo_mapped.shape (259085, 25)
  Elapse_time for "prepare_data_2". It took 7.578682 seconds - 0hh:0mm:7ss.
 start time: Aug 29 2023 02:29:10  end time:  Aug 29 2023 02:29:17
Training starts for test_size = 0.3, c_parm = 0.1 
Done Training 
  Elapse_time for "proc_predict". It took 71.904077 seconds - 0hh:1mm:11ss.
 start time: Aug 29 2023 02:29:17  end time:  Aug 29 2023 02:30:29
Validation for c_parm = 0.1, test_size = 0.3, and embed_type tfidf 
Accuracy Score = 0.8171782929778967
 f1_score_weighted = 0.7876267710987631
 f1_score_macro    = 0.1334649684000625
 f1_score_micro    = 0.8171782929778968
Display metrics
tab_name = tfidf_L1_L2_TM_tst_siz0.3_C0.1
  Elapse_time for "validation". It took 0.237823 seconds - 0hh:0mm:0ss.
 start time: Aug 29 2023 02:30:29  end time:  Aug 29 2023 02:30:30
  Elapse_time for "proc_append". It took 8.983813 seconds - 0hh:0mm:8ss.
 start time: Aug 29 2023 02:30:30  end time: 

tab_name = W2vec_L1_L2_TM_tst_siz0.3_C10
  Elapse_time for "validation". It took 0.255900 seconds - 0hh:0mm:0ss.
 start time: Aug 29 2023 05:05:56  end time:  Aug 29 2023 05:05:56
  Elapse_time for "proc_append". It took 9.727934 seconds - 0hh:0mm:9ss.
 start time: Aug 29 2023 05:05:56  end time:  Aug 29 2023 05:06:06
---------------------------------------------\FINAL COUNTS:
---------------------------------------------
1) 259085
2) 259085
3) 259085
4) 259085
5) 259085
6) 259085
 Elapse_time for "proc_cr_df". It took 4.790055 seconds - 0hh:0mm:4ss.
 start time: Aug 29 2023 05:06:06  end time:  Aug 29 2023 05:06:11
  Elapse_time to build model for for test_size = 0.3, c_parm = 10, embed_typ= W2vec It took 2830.361480 seconds - 0hh:47mm:10ss.
 start time: Aug 29 2023 04:19:00  end time:  Aug 29 2023 05:06:11
Training starts for test_size = 0.3, c_parm = 100 
Done Training 
  Elapse_time for "proc_predict". It took 2802.923455 seconds - 0hh:46mm:42ss.
 start time: Aug 29 2023 05:06:11  

In [ ]:
stop

In [ ]:
excel_file = DBFO + f'L1_L2_syndigo_pred_tst_size{test_size}_082527.xlsx'
save_excel( excel_file, dic_predict)

In [ ]:
dic_predict.items()

In [27]:
test_size = 0.3 
excel_file = DBFO + f'L1_L2_syndigo_pred_cls_rpt_size{test_size}_082527.xlsx'
save_excel_metric( excel_file, dic_metric, l_metric)


excel_file = DBFO + f'L1_L2_syndigo_pred_tst_size{test_size}_082527.xlsx'
save_excel_pred( excel_file, dic_predict)

In [29]:
multi_class = 'multinomial'
score = roc_auc_score(y_test, preds, multi_class= multi_class)
print(f"ROC AUC score for Multiclass= {multi_class}: {score:.2f}")

AxisError: axis 1 is out of bounds for array of dimension 1